This code uses a CSV file that contains the employee records of a fictious company called Softwork Technologies

# Buiding an ETL pipeline
1. Extract data from the CSV file into a Pandas Dataframe
2. Transform the data; ie, clean the data: deal with missing and duplicate data
3. Create a database
4. Load the clean data into the database

In [1]:
#Import libraries
import pandas as pd #For data extraction and transformation(cleaning, manipulation)
import psycopg2 #For connecting python to Postgresql DB
from sqlalchemy import create_engine #To efficiently manage and reuse the DB connection

## Step 1: Extract data from the CSV file into a Pandas Dataframe

In [12]:
data = pd.read_csv("C:\\Users\\......\\Softwork Technologies.csv")#Please provide your own filepath
#Double \\ are used to override escape sequence character that begins with \. Another option is to do read_csv(r"original path"). This converts the 
#path to a raw string. 

In [13]:
data.head() #Displays first 5 rows - ensures that the file has some data

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,61


In [14]:
data.tail() #Displays last 5 rows

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
23485,53478,Legal,region_2,Below Secondary,m,sourcing,1,24,3.0,1,0,61
23486,25600,Technology,region_25,Bachelor's,m,sourcing,1,31,3.0,7,0,74
23487,45409,HR,region_16,Bachelor's,f,sourcing,1,26,4.0,4,0,50
23488,1186,Procurement,region_31,Bachelor's,m,sourcing,3,27,NaN,1,0,70
23489,5973,Technology,region_17,Master's & above,m,other,3,40,5.0,5,0,89


## 2. Transform the data; ie, clean the data: deal with missing and duplicate data

In [16]:
data.duplicated().sum() #To check the number of duplicate records

0

In [17]:
#Explore  missing data
data.isnull()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
0,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
23485,False,False,False,False,False,False,False,False,False,False,False,False
23486,False,False,False,False,False,False,False,False,False,False,False,False
23487,False,False,False,False,False,False,False,False,False,False,False,False
23488,False,False,False,False,False,False,False,False,True,False,False,False


In [18]:
data.isnull().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
awards_won                 0
avg_training_score         0
dtype: int64

In [20]:
#Exploring null education column records
data[data['education'].isnull()].head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
49,3720,Sales & Marketing,region_2,NaN,m,sourcing,1,36,5.0,4,0,49
67,26977,Operations,region_2,NaN,m,sourcing,1,28,3.0,3,0,59
89,2347,Analytics,region_15,NaN,m,other,2,37,5.0,8,0,82
127,42719,Analytics,region_21,NaN,m,sourcing,1,27,NaN,1,0,80
138,2721,Analytics,region_22,NaN,f,sourcing,2,26,5.0,3,0,86


In [21]:
#Exploring null previous year rating column records
data[data['previous_year_rating'].isnull()].head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,awards_won,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,0,77
21,5677,Technology,region_17,Bachelor's,m,sourcing,1,25,NaN,1,0,80
32,67672,Technology,region_17,Bachelor's,m,other,1,29,NaN,1,0,85
39,55325,Analytics,region_22,Bachelor's,m,other,1,25,NaN,1,0,88
47,44159,Analytics,region_22,Master's & above,m,other,1,31,NaN,1,0,84


### Dealing with missing data
WE cannot remove the records with missing data as these are records of existing employees. So, we have to deal with the missing data. In the education column, the NaNs can be replaced with "Unknown" and previous_year_rating to 0.

In [23]:
data['education'].fillna('unknown',inplace=True)
data['previous_year_rating'].fillna(0,inplace=True)

In [24]:
data.isnull().sum()

employee_id             0
department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
awards_won              0
avg_training_score      0
dtype: int64

## 3. Create a database
Go to Postgres PGAdmin4 app and create a DB

In [41]:
#Database Credentials Obtained from Postgresql->properties->connection
username = 'postgres'
password = #Please provide your own password
host = 'localhost'
port = 5432
db_name = 'postgres'

In [42]:
#Establish a connection
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{db_name}')

## 4. Load the clean data to the DB table emp_data in Postgresql

In [43]:
data.to_sql('emp_data',engine,if_exists='replace',index = False)

#Close the connection
engine.dispose()